In [51]:
#Diversos
import pandas as pd
from IPython.display import clear_output
import ToolBox.kit_function as kit

#WebDriver
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait

#Ações
from selenium.webdriver import ActionChains
from selenium.webdriver.common.actions.wheel_input import ScrollOrigin
#from ast import Break

# Exceções
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException as Timeout
from selenium.common.exceptions import ElementNotInteractableException as NotInteract
from selenium.common.exceptions import ElementClickInterceptedException as NotAvailibleElement
from selenium.common.exceptions import NoSuchWindowException
from selenium.webdriver.support import expected_conditions as EC

In [52]:
# Inicializa a instância do EDGE
service = Service()

options = webdriver.EdgeOptions()
options.add_argument("-inprivate")
driver = webdriver.Edge(service=service, options=options)
driver.maximize_window()

url = 'https://toolsworldb2b.meuspedidos.com.br/'

driver.get(url)

In [53]:
sheetCategorias = 'Links/CategoriasMTX.xlsx'
columns = ['Link']
categoriasLinks = kit.readExcel(sheetCategorias,columns)

In [54]:
#Título do Cartão do Produto
cssCategoryTitle = '#catalogWrapper > div:nth-child(2) > div.Catalog__titleB2bContainer___JtAi4 > div > div.Catalog__titleB2bInner___G8ViQ'

#Titulo do produto
clsCardTitle = 'catalog-item-b2b-title'

#Dados do Produto
clsNomeProd = 'ProductInfo__productNameWithAddedToCardIcon___CTBpv'
clsCodProd = 'ProductInfo__productInfo___rlEL5'
clsPrcProd = 'AddToCartContainer__informationForSale___1DaML'
cssMaisInfo = "i[class='materialIcons__materialIcons___NHWaT PopoverPriceDetails__icon___r5kg2 PopoverPriceDetails__container___Gki+K']"
xpathImpostoProd = '/html/body/div[4]/div/ul[4]/li[2]'
cssSeletorMoreItens = "button[class='b2b-button Catalog__links___4450R Catalog__buttonSeeMoreProducts___kZGUm Button__button___Oss6- Button__pointer___mYfJr Button__info___4h9Qg Button__medium___FQv5P Button__buttonInfo___aqiZj custom-brand-full-b2b']"

In [ ]:
init = 0
for posicao, link in enumerate(categoriasLinks['Link'][init:len(categoriasLinks['Link'])], start=init):
    kit.wait(10)
    clear_output(wait=False)

    print(f'Posição: {posicao}, Link: {link}')
    driver.get(link)
    
    #------------------------------Captura o título da categoria------------------------------
    titleCategoria = kit.safe_find_element(driver,By.CSS_SELECTOR , cssCategoryTitle).text
    print(titleCategoria)
    
    
    #-----------------------Verifica a quantidade de itens na página--------------------------
    kit.VerifyElement(By.CLASS_NAME, clsCardTitle,driver)
    itens = driver.find_elements(By.CLASS_NAME, clsCardTitle)
    print('Iniciando varredura, quantidade inicial:', len(itens))

    #-----------------------Scroll-----------------------
    try:
        while True:
            kit.VerifyElement(By.CSS_SELECTOR, cssSeletorMoreItens,driver)
            verProdutos = driver.find_element(By.CSS_SELECTOR, cssSeletorMoreItens)
            verProdutos.click()
    except:
            print('Verificação Terminada.')
            #Break()
            
    kit.rollpage(driver,By.TAG_NAME, "footer")

    #-----------------------Seta a quantidade de itens na pagina-----------------------
    itens = driver.find_elements(By.CLASS_NAME, clsCardTitle)
    print('Total: ' + str(len(itens)) + ' Itens')
    
    
    #-----------------------Captura dos Dados-----------------------
    #Array de Dados
    nomeList= []
    codList = []
    prcList = []
    impList = []
    prodUrlList = []

    #Variavel contadora de itens pulados
    iJump = 0
    
    #Controle de posição
    prod = 0
    
    #Método para percorrer a página
    while prod < len(itens):
        try:
            #Espera carregar
            kit.VerifyElement(By.CLASS_NAME, clsCardTitle,driver)
            itens = driver.find_elements(By.CLASS_NAME,clsCardTitle)
            print('Prox item n°: ' , prod , ' ', itens[prod].text)
            
            #Vai ao item
            driver.execute_script("arguments[0].scrollIntoView(true);", itens[prod])
            #Ajuste fino
            kit.rollpage(driver,By.TAG_NAME, "footer", -350)
            
            itens[prod].click()
            
        except (NotInteract,NotAvailibleElement):
            print('Produto Não Disponivel N°: ',iJump ,'. Incrementando')
            prod += 1
            iJump +=1
            print('Not found:' , itens[prod].text)
            itens[prod].click()           
        
        kit.wait(10)
        #---------------------------Captura de Dados----------------------
        # Captura o nome 
        nome = kit.safe_find_element(driver, By.CLASS_NAME, clsNomeProd, 'Nome não encontrado')

        # Captura o código
        cod = kit.safe_find_element(driver, By.CLASS_NAME, clsCodProd, 'Código não encontrado')

        # Captura o preço
        prc = kit.safe_find_element(driver, By.CLASS_NAME, clsPrcProd, "erro")

        # Captura o imposto
        try:
            hoverable = driver.find_element(By.CSS_SELECTOR, cssMaisInfo)
            ActionChains(driver).move_to_element(hoverable).perform()
            imp = driver.find_element(By.XPATH, xpathImpostoProd).text
        except:
            imp = "erro"

        if nome == 'Nome não encontrado':
            prod-=1
        else:     
            nomeList.append(nome if isinstance(nome, str) else nome.text)
            codList.append(cod if isinstance(cod, str) else cod.text)
            prcList.append(prc if isinstance(prc, str) else prc.text)
            impList.append(imp)
            prodUrlList.append(driver.current_url)
        
        prod +=1    
        driver.back()

    df = {'Nome':nomeList,
        'codigo': codList,
        'prc': prcList,
        'imp': impList,
        'link':prodUrlList}

    xlsx_dir = 'Dados/'

    mtxTable = pd.DataFrame(df)

    file_name = xlsx_dir + titleCategoria + '.xlsx'

    mtxTable.to_excel(file_name, index=False)
    print('Itens da Categoria: ' + titleCategoria + ' Foram salvos')

Posição: 13, Link: https://toolsworldb2b.meuspedidos.com.br/?representada=448126&categoria=1740434
Esperando Elemento...
Elemento encontrado!!!
BROCA
Esperando Elemento...
Elemento encontrado!!!
Iniciando varredura, quantidade inicial: 48
Esperando Elemento...
Elemento encontrado!!!
Esperando Elemento...
Elemento encontrado!!!
Esperando Elemento...
Elemento encontrado!!!
Esperando Elemento...
Elemento encontrado!!!
Esperando Elemento...
Elemento encontrado!!!
Esperando Elemento...
Elemento encontrado!!!
Esperando Elemento...
Elemento encontrado!!!
Esperando Elemento...
Elemento encontrado!!!
Esperando Elemento...
Elemento encontrado!!!
Esperando Elemento...
Elemento encontrado!!!
Esperando Elemento...
O tempo de espera para o elemento expirou.
Verificação Terminada.
Total: 499 Itens
Esperando Elemento...
Elemento encontrado!!!
Prox item n°:  0   BROCA ACO RAPIDO PROFISSIONAL, COBALTO 5%, METAL DURO E ACO INOX, 10MM, 1PC // MTX
Esperando Elemento...
Elemento encontrado!!!
Esperando Elem

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: MicrosoftEdge=131.0.2903.146)
Stacktrace:
	(No symbol) [0x00007FF6570E6B05]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF65740F444+1437348]
	sqlite3_dbdata_init [0x00007FF6574B2D76+643174]
	(No symbol) [0x00007FF656FA7657]
	(No symbol) [0x00007FF657031CE5]
	(No symbol) [0x00007FF65704670A]
	(No symbol) [0x00007FF65702BE03]
	(No symbol) [0x00007FF657002984]
	(No symbol) [0x00007FF657001E30]
	(No symbol) [0x00007FF657002571]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF6573BBAD4+1094964]
	(No symbol) [0x00007FF6571232B8]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF6573BAF13+1091955]
	Microsoft::Applications::Events::EventProperty::empty [0x00007FF6573BAA79+1090777]
	Microsoft::Applications::Events::ILogConfiguration::operator* [0x00007FF6571C0C81+461569]
	Microsoft::Applications::Events::ILogConfi

In [48]:
df = {'Nome':nomeList,
    'codigo': codList,
    'prc': prcList,
    'imp': impList,
    'link':prodUrlList}

xlsx_dir = 'Dados/'

mtxTable = pd.DataFrame(df)

file_name = xlsx_dir + titleCategoria + '.xlsx'

mtxTable.to_excel(file_name, index=False)
print('Itens da Categoria: ' + titleCategoria + ' Foram salvos')

Itens da Categoria: BROCA Foram salvos
